In [1]:
!pip install --quiet -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [5]:
# Load job titles data 
titles_df = pd.read_csv("/kaggle/input/title-matching/lightcast-titles.csv")
titles = titles_df["Title"].tolist()

candidates_df = pd.read_excel('/kaggle/input/title-matching/jd-candidate-data-dropna.xlsx')

candidates_df.head()

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,job_categories,designation,job_id,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
0,ACCEPTED,2.00,3.6,4.14,15,Senior Sales Executive,"Inside Sales (Outbound),Sales",Inside Sales,14,0.5,3.0,3.0,5.0
1,REJECTED,1.42,6.0,6.90,15,Customer Service Associate|||Senior Business D...,"Inside Sales (Outbound),Sales",Inside Sales,14,0.5,3.0,3.0,5.0
2,ACCEPTED,11.00,3.0,4.00,1,Marketing executive,"Inside Sales (Outbound),Sales",Inside Sales,14,0.5,3.0,3.0,5.0
3,ACCEPTED,2.50,5.0,5.75,15,Android Developer,Frontend Developer,React Native Developer,3,2.0,4.0,6.0,10.0
4,ACCEPTED,4.08,7.0,8.05,15,Android Mobile Application Developer|||Android...,Frontend Developer,React Native Developer,3,2.0,4.0,6.0,10.0


In [6]:
# Assuming 'df_candidates' and 'df_titles' are your DataFrames
# Encode candidate profiles, designation
candidate_profiles = candidates_df["candidate_profiles"].tolist()
jd_designations = candidates_df["designation"].tolist()

candidate_embeds = model.encode(candidate_profiles,show_progress_bar=False)
designation_embeds = model.encode(jd_designations,show_progress_bar=False)

# Encode titles dataset  
title_embeds = model.encode(titles, show_progress_bar=False)

In [7]:
# Cosine Similarity
candidate_cosine_sim = cosine_similarity(candidate_embeds, title_embeds)
jd_cosine_sim = cosine_similarity(designation_embeds,title_embeds)

# Get the index of the closest matches
cosine_best_match_candidate = candidate_cosine_sim.argmax(axis=1)
cosine_best_match_designation = jd_cosine_sim.argmax(axis=1)

In [8]:
# Get best matched titles
matched_titles_candidates = [titles[idx] for idx in cosine_best_match_candidate]
matched_titles_designation = [titles[idx] for idx in cosine_best_match_designation]


# Add to df
candidates_df['norm_profile'] = matched_titles_candidates
candidates_df['norm_designation'] = matched_titles_designation

In [9]:
# Reorder columns
columns_reordered = ['status', 'experience', 'present_ctc', 'expected_ctc', 'notice_period',
                     'candidate_profiles','norm_profile', 'job_categories', 
                     'designation', 'norm_designation','Relevant', 'exp_min', 'exp_max', 'ctc_fixed_min', 
                     'ctc_fixed_max']

# Reindex the DataFrame with the reordered columns
candidates_df = candidates_df.reindex(columns=columns_reordered)

candidates_df.head()  # Displaying the reordered DataFrame

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
0,ACCEPTED,2.00,3.6,4.14,15,Senior Sales Executive,Sales Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,NaN,0.5,3.0,3.0,5.0
1,REJECTED,1.42,6.0,6.90,15,Customer Service Associate|||Senior Business D...,Customer Development Associate,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,NaN,0.5,3.0,3.0,5.0
2,ACCEPTED,11.00,3.0,4.00,1,Marketing executive,Marketing Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,NaN,0.5,3.0,3.0,5.0
3,ACCEPTED,2.50,5.0,5.75,15,Android Developer,Android Developer,Frontend Developer,React Native Developer,React Native Developer,NaN,2.0,4.0,6.0,10.0
4,ACCEPTED,4.08,7.0,8.05,15,Android Mobile Application Developer|||Android...,React Native Developer,Frontend Developer,React Native Developer,React Native Developer,NaN,2.0,4.0,6.0,10.0


In [10]:
candidates_df.to_excel("all-miniLM-L12-v2_norm-dataset.xlsx", index=False)

In [11]:
candidates_df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
13197,REJECTED,4.5,6.50,12.0,60,Data Analyst,Data Analyst,Analytics,Management Information SystemAnalyst,Management Information Systems Analyst,NaN,1.0,3.0,10.0,12.0
6691,REJECTED,10.0,1500000.00,600000.0,1,System Administrator,Systems Administrator,Engineering,System Administrator,Systems Administrator,NaN,2.0,4.0,5.0,6.0
4455,ACCEPTED,9.0,13.00,15.6,60,Deputy Manager- Finance & Accounts|||Manager- ...,Deputy Finance Manager,Finance,Head Finance,Head of Finance,NaN,8.0,12.0,13.0,22.0
5498,ACCEPTED,5.0,9.50,11.0,30,Associate Product Manager,Associate Product Manager,Client operations,Solution Engineer/PreSales Engineer,Pre-Sales Solutions Engineer,NaN,1.0,3.0,9.0,10.0
7531,ACCEPTED,4.5,14.00,20.0,30,Devops engineer,DevOps Engineer,Developer,DevOps Engineer,DevOps Engineer,NaN,3.0,5.0,15.0,25.0
17126,ACCEPTED,5.0,7.00,9.0,30,Assistant Pre-Sales Manager|||Assistant Sales ...,Sales Executive/Business Development Manager,"Lead generation,Sales",PreSales Executive/ Manager,Pre-Sales Manager,NaN,1.0,8.0,6.0,12.0
3011,ACCEPTED,3.0,5.25,6.0,30,Assistant Manager Marketing,Assistant Marketing Manager,Key account manager/ Customer success manager,GEM Expert,Gemologist,NaN,1.0,5.0,2.4,3.0
14472,ACCEPTED,14.0,26.00,32.0,30,Business Development Manager|||SALES & Managem...,Sales Executive/Business Development Manager,Business development,Sales Manager - Mumbai,Sales Manager,NaN,2.0,15.0,8.0,14.0
20927,ACCEPTED,2.5,3.36,4.5,7,Account Manager|||E Commerce Account Manager,E-Commerce Account Manager,Sales,Account Manager,Account Manager,NaN,2.0,4.0,4.0,5.0
18623,REJECTED,3.0,7.00,8.0,30,Business Development Executive|||Manager - ten...,Marketing and Business Development Executive,Business development,Sales manager - NCR,National Sales Manager,NaN,3.0,9.0,7.0,11.0
